## 1. Persiapan

In [ ]:
# import library yang dibutuhkan
import sqlite3
import pandas as pd

In [ ]:
from google.colab import drive

In [ ]:
# cek versi dari sqlite kita
sqlite3.sqlite_version

'3.37.2'

Ada 2 cara untuk membuat koneksi ke database

In [ ]:
# membuat koneksi ke database via path
conn = sqlite3.connect('/content/northwind.db')

In [ ]:
# membuat koneksi ke database via upload file
from google.colab import files
uploaded = files.upload()

In [ ]:
conn = sqlite3.connect('northwind.db')

## 2. Mengakses Database

In [ ]:
# buat variabel cursor dari koneksi ke database kita
cursor = conn.cursor()

In [ ]:
# tuliskan query yang akan kita jalankan
# query di bawah akan menampilkan tabel - tabel yang ada di database kita
query = """
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';
"""

In [ ]:
# menjalankan query dan menampilkan hasil
cursor.execute(query)
print(cursor.fetchall())

[('Suppliers',), ('Shippers',), ('Employees',), ('Customers',), ('Orders',), ('Categories',), ('Products',), ('OrderDetails',)]


# 3. Study Case

1. Tampilkan data dari tabel customers dengan tujuan kota London dengan title sebagai Sales Representative

2. Tampilkan data dari tabel categories yang mengandung coffees di dalamnya

3. Tampilkan data dari tabel products yang masih ada (belum discontinued) dengan rentang harga antara 10 sampai 15

4. Hitung total pendapatan dari tabel OrderDetails untuk semua produk yang belum di-discontinue tersebut pada tabel Products, dengan mempertimbangkan jumlah unit yang terjual (Quantity) dan harga sebenarnya (UnitPrice) setelah diskon (Discount)

5. Berapakah harga termurah dengan productid 46 jika semuanya mendapatkan diskon sebesar 27% pada tabel orderdetail

6. Tampilkan Top 3 Supplier berdasarkan jumlah produk yang mereka sediakan dari tabel Suppliers dan Products. Hanya tampilkan SupplierID, Nama Supplier, dan jumlah produk yang mereka sediakan

# 4. Jawaban

In [ ]:
# 1
query_1 = """

SELECT * FROM customers where ContactTitle = "Sales Representative" and City = "London"

"""
pd.read_sql_query(query_1, conn)

,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
1,BSBEV,B's Beverages,Victoria Ashworth,Sales Representative,Fauntleroy Circus,London,None,EC2 5NT,UK,(171) 555-1212,None
2,CONSH,Consolidated Holdings,Elizabeth Brown,Sales Representative,Berkeley Gardens 12 Brewery,London,None,WX1 6LT,UK,(171) 555-2282,(171) 555-9199


In [ ]:
# 2
query_2 = """

SELECT * FROM categories where Description LIKE "%coffees%"

"""
pd.read_sql_query(query_2, conn)

,CategoryID,CategoryName,Description,Picture
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales","b""\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\..."


In [ ]:
# 3
query_3 = """

SELECT * FROM products Where Discontinued = 0 AND UnitPrice BETWEEN 10 AND 15

"""
pd.read_sql_query(query_3, conn)

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
1,21,Sir Rodney's Scones,8,3,24 pkgs. x 4 pieces,10.00,3,40,5,0
2,25,NuNuCa Nuß-Nougat-Creme,11,3,20 - 450 g glasses,14.00,76,0,30,0
3,31,Gorgonzola Telino,14,4,12 - 100 g pkgs,12.50,0,70,20,0
4,34,Sasquatch Ale,16,1,24 - 12 oz bottles,14.00,111,0,15,0
5,46,Spegesild,21,8,4 - 450 g glasses,12.00,95,0,0,0
6,48,Chocolade,22,3,10 pkgs.,12.75,15,70,25,0
7,58,Escargots de Bourgogne,27,8,24 pieces,13.25,62,0,20,0
8,67,Laughing Lumberjack Lager,16,1,24 - 12 oz bottles,14.00,52,0,10,0
9,68,Scottish Longbreads,8,3,10 boxes x 8 pieces,12.50,6,10,15,0


In [ ]:
# 4


query_4 = """

SELECT SUM((OrderDetails.UnitPrice * (1 - OrderDetails.Discount)) * OrderDetails.Quantity) AS Pendapatan
FROM OrderDetails
INNER JOIN Products ON OrderDetails.ProductID = Products.ProductID
WHERE Discontinued = 0 ;

"""
pd.read_sql_query(query_4, conn)

,Pendapatan
0,1.081230e+06


In [ ]:
# 5
query_5 = """

SELECT MIN(UnitPrice * (1 - 0.27))
FROM OrderDetails
WHERE ProductID = 46

"""
pd.read_sql_query(query_5, conn)

,MIN(UnitPrice * (1 - 0.27))
0,7.008


In [ ]:
query_5 = """

SELECT * FROM Products

"""
pd.read_sql_query(query_5, conn)

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
72,73,Röd Kaviar,17,8,24 - 150 g jars,15.00,101,0,5,0
73,74,Longlife Tofu,4,7,5 kg pkg.,10.00,4,20,5,0
74,75,Rhönbräu Klosterbier,12,1,24 - 0.5 l bottles,7.75,125,0,25,0
75,76,Lakkalikööri,23,1,500 ml,18.00,57,0,20,0


In [ ]:
query_6 = """

SELECT s.SupplierID, s.CompanyName, COUNT(p.ProductID) AS Total
FROM Suppliers s
INNER JOIN Products p ON  s.SupplierID = p.SupplierID
GROUP BY s.SupplierID, s.CompanyName
ORDER BY Total DESC
LIMIT 3



"""
pd.read_sql_query(query_6, conn)

,SupplierID,CompanyName,Total
0,7,"Pavlova, Ltd.",5
1,12,Plutzer Lebensmittelgroßmärkte AG,5
2,2,New Orleans Cajun Delights,4
